In [2]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pickle
import ast
import pandas
from numpy import array  
import nbimporter
import auto_encoder as aec
np.set_printoptions(suppress=True)

Importing Jupyter notebook from auto_encoder.ipynb


In [3]:
def column(matrix, i):
    return [row[i] for row in matrix]

In [ ]:
# Linking Id's
'''
project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\ratings.csv")
rating = pandas.read_csv(data_path)

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\links.csv")
links = pandas.read_csv(data_path)

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\movies_metadata.csv")
movie_meta = pandas.read_csv(data_path)

joined = links.set_index('tmdbId').join(movie_meta.set_index('id'))

writer = pandas.ExcelWriter('linked_movieIds.xlsx')
joined.to_excel(writer,'Sheet1')
writer.save()
'''

In [ ]:
# Removing less-voted movies & least frequent voters from ratings.csv
'''
project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\ratings.csv")
rating = pandas.read_csv(data_path, engine='python')

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\movie_votecount.csv")
vote_count = pandas.read_csv(data_path)

votes = column(vote_count.values, 0)
ratings_updated = rating.loc[rating["movieId"].isin(votes)]

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\user_votecount.csv")
user_vote = pandas.read_csv(data_path)

votes = column(user_vote.values, 0)
ratings_updated = ratings_updated.loc[ratings["userId"].isin(votes)]

file = open("ratings_updated", "wb")
pickle.dump(ratings_updated, file)
'''

In [10]:
file = open("ratings_updated", "rb")
rating = pickle.load(file)

In [8]:
# Removing less-voted movies & least frequent voters from ratings.csv
'''
project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\movie_votecount2.csv")
movie_vote = pandas.read_csv(data_path)
votes = column(movie_vote.values, 0)
ratings_updated = rating.loc[rating["movieId"].isin(votes)]

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\user_votecount2.csv")
user_vote = pandas.read_csv(data_path)

votes = column(user_vote.values, 0)
ratings_updated = ratings_updated.loc[rating["userId"].isin(votes)]
file = open("ratings_updated_2", "wb")
pickle.dump(ratings_updated, file)
'''

In [ ]:
file = open("ratings_updated_2", "rb")
ratings_updated = pickle.load(file)

In [ ]:
# Movie matrix build
movies = ratings_updated["movieId"]
movies = set(movies)
movies = sorted(movies, key=lambda x: int(x), reverse=False)

In [ ]:
# User matrix build
users = ratings_updated["userId"]
users = set(users)
users = sorted(users, key=lambda x: int(x), reverse=False)

In [ ]:
# User-movie matrix build
user_movie = np.zeros((len(users),len(movies)), dtype=float)

project_root = os.path.dirname(os.path.abspath(""))
data_path = os.path.join(project_root, "data\\movies.csv")
movies_meta = pandas.read_csv(data_path, encoding = "ISO-8859-1")

# Fill non-rated movies as the average rating of that movie
for i in range(len(movies)):
    rating_of_i = (movies_meta.loc[movies_meta["movieId"] == movies[i]])["vote_average"].values[0]
    x = np.full((len(users),), rating_of_i / 2, dtype=float)
    user_movie[:,i] = x

for index, row in ratings_updated.iterrows():
    user_movie[users.index(row['userId'])][movies.index(row['movieId'])] = row['rating']
    
print(np.shape(user_movie))
file = open("user_movie", "wb")
pickle.dump(user_movie, file)

In [ ]:
# Load user-movie matrix
file = open("user_movie", "rb")
user_movie = pickle.load(file)

In [ ]:
# Train the Autoencoder to get user features
neuron_list = [499,250,125,250,499]
nn = aec.Autoencoder(neuron_list, 0.002, 300)
inp = np.transpose([np.transpose(user_movie)])
print(np.shape(inp))
err = nn.train(inp,inp, 10)
plt.plot(err)
plt.show()
print(err)

In [ ]:
# Extract user-features after training
features = []
for i in range(len(user_movie)):
    encoded = nn.encode(inp[i])
    features.append(encoded)

user_features = []
for i in range(len(users)):
    user_features.append([users[i], features[i]])
file = open("user_features", "wb")
pickle.dump(user_features, file)